# EXERCISE 5.3: Cloud masking using Google Earth Engine 

---

**Use of Google Earth Engine (API) in python to visualize geodata:**
* a) Identify cloud cover components and cloud shadows in EO data  
* b) Process cloud cover  
* c) Remove cloud cover from map 

This tutorial is an introduction to masking clouds and cloud shadows in Sentinel-2 surface reflectance (SR) data using Google Earth Engine (GEE). Clouds are identified from the Sentinel 2 cloud probability dataset (s2cloudless) and shadows are defined by cloud projection intersection with low-reflectance near-infrared (NIR) pixels.

### Run me first

Run the following cell to initialize the Earth Engine API. The output will contain instructions on how to grant this notebook access to Earth Engine using your account.


1.   Run the cell below and you will be prompted to enter a verification code.
2.   Follow the link below.
3.   Make sure the google account is the account you created your ML4EO project with
4. Make sure the project ML4EO is selected (you already set this up in 5_02)
5. Click "Generate Token"
6. Click on the Google account you are running this notebook from.
7. Click continue
8. In the page that appears, ensure "View and manage your Google Earth Engine data." and "Manage your data and permissions in Cloud Storage and see the email address for your Google Account." are both checked.
9. Copy the code you see under "Authorization Code" and paste it in the prompt below the cell.

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=71ZWEhRV47GvtaLOmS-iM6JvKugUt0MtF_L-JkIeHU4&tc=DOhikcSqPWnP4uN5D-u7ty2CIWF2yMzA_RYUpnf20Xc&cc=-ouSBOpnOuLPrSStfvsVxc_ioPMGXie5JDhtCDuPlxM

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk4lbJfKWsNWdM2iRezCU8IBhiO0OAFNO664Zc9U56gjwawjMqDnfeM

Successfully saved authorization token.


## Assemble cloud mask components

This section builds a Sentinel 2 surface reflectivity collection and defines functions to add cloud and cloud shadow component layers to each image in the collection.



### Define collection filter and cloud mask parameters

We first define parameters that will be used to filter the Sentinel 2 image collection and determine cloud and cloud shadow identification.

|Parameter | Type | Description |
| :-- | :-- | :-- |
| `AOI` | `ee.Geometry` | Area of interest |
| `START_DATE` | string | Image collection start date (inclusive) |
| `END_DATE` | string | Image collection end date (exclusive) |
| `CLOUD_FILTER` | integer | Maximum image cloud cover percent allowed in image collection |
| `CLD_PRB_THRESH` | integer | Cloud probability (%); values greater than are considered cloud |
| `NIR_DRK_THRESH` | float | Near-infrared reflectance; values less than are considered potential cloud shadow |
| `CLD_PRJ_DIST` | float | Maximum distance (km) to search for cloud shadows from cloud edges |
| `BUFFER` | integer | Distance (m) to dilate the edge of cloud-identified objects |

The values currently set for `AOI`, `START_DATE`, `END_DATE`, and `CLOUD_FILTER` are intended to build a collection for a single Sentinel 2 overpass of a region near Kigali, Rwanda. When parameterizing and evaluating cloud masks for a new area, it is good practice to identify a single overpass date and limit the regional extent to minimize processing requirements. If you want to work with a different example, use this [Earth Engine App](https://showcase.earthengine.app/view/s2-sr-browser-s2cloudless-nb) to identify an image that includes some clouds, then replace the relevant parameter values below with those provided in the app.


In [ ]:
AOI = ee.Geometry.Point(30.0619, -1.96)
START_DATE = '2023-02-16'
END_DATE = '2023-02-17'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

### Build a Sentinel-2 collection

[Sentinel-2 surface reflectance](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR) and [Sentinel-2 cloud probability](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_CLOUD_PROBABILITY) are two different image collections. Each collection must be filtered similarly (e.g., by date and bounds) and then the two filtered collections must be joined.

Let's define a function to filter the Sentinel 2 and s2cloudless collections according to area of interest and date parameters, then join them on the `system:index` property. The result will be a copy of the Sentinel 2 collection where each image has a new `'s2cloudless'` property whose value is the corresponding s2cloudless image.

The Sentinel 2 cloud probability dataset was created with the sentinel2-cloud-detector library (using LightGBM). All bands are upsampled using bilinear interpolation to 10m resolution before the gradient boosting algorithm is applied. The resulting 0..1 floating point probability is scaled to 0..100 and stored as a UINT8. Areas missing any or all of the bands are masked out. Higher values are more likely to be clouds or highly reflective surfaces (e.g. roof tops or snow).

The Sentinel 2 cloud probability dataset contains a single band:
<table class="eecat">
<tbody><tr>
<th scope="col">Name</th>
<th scope="col">Min</th>
<th scope="col">Max</th>
<th scope="col">Pixel Size</th>
<th scope="col">Description</th>
</tr>
<tr>
<td><code translate="no" dir="ltr">probability</code></td>
<td>
          0
</td>
<td>
          100
</td>
<td>
      10 meters
</td>
<td><p>Probability that the pixel is cloudy.</p></td>
</tr>
</tbody></table>

Now let's define a function to help extract our data:

In [ ]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

Let's apply the `get_s2_sr_cld_col` function to build a collection according to the parameters defined above.

In [ ]:
s2_sr_cld_col_eval = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

### Define cloud mask component functions

#### Cloud components

Define a function to add the s2cloudless probability layer and derived cloud mask as bands to an Sentinel 2 SR image input.

In [ ]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

#### Cloud shadow components

Define a function to add dark pixels, cloud projection, and identified shadows as bands to an Sentinel 2 SR image input. Note that the image input needs to be the result of the above `add_cloud_bands` function because it relies on knowing which pixels are considered cloudy (`'clouds'` band).

In [ ]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)   # An SCL of 6 indicates water

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    distance_to_cloud = img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)

    # Rescale the existing projection
    reprojected = distance_to_cloud.reproject(**{'crs': img.select(0).projection(), 'scale': 100})

    # Use the distance band (returned by directionalDistanceTransform) as mask
    cld_proj = reprojected.select('distance').mask().rename('cloud_transform')

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

## <font color=orange>Discussion:</font>
In the function above, the CLD_PRJ_DIST determines the maximum distance over which a cloud corresponding to a shadow is searched. 
How can increasing/decreasing CLD_PRJ_DIST affect the computational time? 
How does increasing/decreasing CLD_PRJ_DIST affect the number of detected shadows?


#### Final cloud-shadow mask

Define a function to assemble all of the cloud and cloud shadow components and produce the final mask.

In [ ]:
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

## Question 5.3.1
Identify the parts of the image loaded (one_image) in the image below that are assigned to Vegetation by the Scene Classificaiton Map band by returning a mask.


In [ ]:
one_image = s2_sr_cld_col_eval.first()
# Solution Here


NameError: ignored

## Visualize and evaluate cloud mask components

This section provides functions for displaying the cloud and cloud shadow components. In most cases, adding all components to images and viewing them is unnecessary. This section is included to illustrate how the cloud/cloud shadow mask is developed and demonstrate how to test and evaluate various parameters, which is helpful when defining masking variables for an unfamiliar region or time of year.

In applications outside of this tutorial, if you prefer to include only the final cloud/cloud shadow mask along with the original image bands, replace:

```
return img_cloud_shadow.addBands(is_cld_shdw)
```

with

```
return img.addBands(is_cld_shdw)
```

in the above `add_cld_shdw_mask` function.

### Define functions to display image and mask component layers.

The python pacakge Folium will be used to display map layers. Import `folium` and define a method to display Earth Engine image tiles.

In [ ]:
# Import the folium library.
import folium

# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

Define a function to display all of the cloud and cloud shadow components to an interactive Folium map. The input is an image collection where each image is the result of the `add_cld_shdw_mask` function defined previously.

In [ ]:
def display_cloud_layers(col):
    # Mosaic the image collection.
    img = col.mosaic()

    # Subset layers and prepare them for display.
    clouds = img.select('clouds').selfMask()
    shadows = img.select('shadows').selfMask()
    dark_pixels = img.select('dark_pixels').selfMask()
    probability = img.select('probability')
    cloudmask = img.select('cloudmask').selfMask()
    cloud_transform = img.select('cloud_transform')

    # Create a folium map object.
    center = AOI.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=12)

    # Add layers to the folium map.
    m.add_ee_layer(img,
                   {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                   'S2 image', True, 1, 9)
    m.add_ee_layer(probability,
                   {'min': 0, 'max': 100},
                   'probability (cloud)', False, 1, 9)
    m.add_ee_layer(clouds,
                   {'palette': 'e056fd'},
                   'clouds', False, 1, 9)
    m.add_ee_layer(cloud_transform,
                   {'min': 0, 'max': 1, 'palette': ['white', 'black']},
                   'cloud_transform', False, 1, 9)
    m.add_ee_layer(dark_pixels,
                   {'palette': 'orange'},
                   'dark_pixels', False, 1, 9)
    m.add_ee_layer(shadows, {'palette': 'yellow'},
                   'shadows', False, 1, 9)
    m.add_ee_layer(cloudmask, {'palette': 'orange'},
                   'cloudmask', True, 0.5, 9)

    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)

### Display mask component layers

Map the `add_cld_shdw_mask` function over the collection to add mask component bands to each image, then display the results.

Give the system some time to render everything, it should take less than a minute.

In [ ]:
s2_sr_cld_col_eval_disp = s2_sr_cld_col_eval.map(add_cld_shdw_mask)

display_cloud_layers(s2_sr_cld_col_eval_disp)

### Evaluate mask component layers

In the above map, use the layer control panel in the upper right corner to toggle layers on and off; layer names are the same as band names, for easy code referral. Note that the layers have a minimum zoom level of 9 to avoid resource issues that can occur when visualizing layers that depend on the `ee.Image.reproject` function (used during cloud shadow project and mask dilation).

## Question 5.3.2
Try changing the above `CLD_PRB_THRESH`, `NIR_DRK_THRESH`, `CLD_PRJ_DIST`, and `BUFFER` input variables and rerunning the previous cell to see how the results change. Experiment with at least 5 sets of `CLD_PRB_THRESH`, `NIR_DRK_THRESH`, `CLD_PRJ_DIST` tuples, take a screenshot of the output of each one and submit your results. Each image should be named `<CLD_PRB_THRESH>-<NIR_DRK_THRESH>-<CLD_PRJ_DIST>.png`

## Apply cloud and cloud shadow mask

In this section we'll generate a cloud-free composite for the same region as above that represents mean reflectance for July and August, 2020.

### Define collection filter and cloud mask parameters

We'll redefine the parameters to be a little more aggressive, i.e. decrease the cloud probability threshold, increase the cloud projection distance, and increase the buffer. These changes will increase cloud commission error (mask out some clear pixels), but since we will be compositing images from three months, there should be plenty of observations to complete the mosaic.

In [ ]:
AOI = ee.Geometry.Point(30.0619, -1.96)
START_DATE = '2023-02-16'
END_DATE = '2023-02-17'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 40
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100

### Build a Sentinel-2 collection

Reassemble the S2-cloudless collection since the collection filter parameters have changed.

In [ ]:
s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

### Define cloud mask application function

Define a function to apply the cloud mask to each image in the collection.

In [ ]:
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

### Process the collection

Add cloud and cloud shadow component bands to each image and then apply the mask to each image. Reduce the collection by median (in your application, you might consider using medoid reduction to build a composite from actual data values, instead of per-band statistics. See this [paper](https://doi.org/10.3390/rs5126481) for more details).

In [ ]:
s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())

### Display the cloud-free composite

Display the results. Be patient while the map renders, it may take a minute; [`ee.Image.reproject`](https://developers.google.com/earth-engine/guides/projections#reprojecting) is forcing computations to happen at 100 and 20 m scales (i.e. it is not relying on appropriate pyramid level [scales for analysis](https://developers.google.com/earth-engine/guides/scale#scale-of-analysis)). The issues with `ee.Image.reproject` being resource-intensive in this case are mostly confined to interactive map viewing. 




In [ ]:
# Create a folium map object.
center = AOI.centroid(10).coordinates().reverse().getInfo()
m = folium.Map(location=center, zoom_start=12)
display(m)
# Add layers to the folium map.
m.add_ee_layer(s2_sr_median,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 cloud-free mosaic', True, 1, 9)

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

## <font color=orange>Discussion:</font> 
What are your observations about the resulting map?

## Cloud Masking: Another Approach
Many datasets include a bit mask that indicates which pixels contain clouds. In the `COPERNICUS/S2_HARMONIZED` dataset, there is a band named QA60 that can be used to mask clouds. Let us see how we can use QA60 to mask bits.

## Question 5.3.3
Load the `COPERNICUS/S2_HARMONIZED` image collection and assign it to a variable named `s2`. 
Filter the image collection to keep images from 2017 only and assign it to a variable named `filtered`.

In [ ]:
# solution


In [ ]:
geometry = ee.Geometry.Polygon([[
  [82.60642647743225, 27.16350437805251],
  [82.60984897613525, 27.1618529901377],
  [82.61088967323303, 27.163695288375266],
  [82.60757446289062, 27.16517483230927]
]])

# Keep images that are in the region defined above only
filtered = filtered.filterBounds(geometry)

# Keep images with CLOUDY_PIXEL_PERCENTAGE less than 30%
filtered = filtered.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))


## Question 5.3.4
Use the comments to complete the function below

In [ ]:
def masks2clouds(img):
  # Select the QA60 band
  
  # A 1 at bit position 10 of QA60 band indicates a cloud
  cloud_bitmask = 1 << 10
  # Generate a mask for points that are not cloud
  mask_non_cloud = qa60.bitwiseAnd(cloud_bitmask).eq(0)

  # TODO: A 1 at bit position 11 of QA60 band indicates a cirrus
  cirrus_bitmask = 
  # TODO: Generate a mask for points that are not cirrus
  mask_non_cirrus = 

  # Combine the two 
  mask = mask_non_cloud.And(mask_non_cirrus)


  return img.updateMask(mask).multiply(0.0001).select('B.*').copyProperties(img, ['system:time_start'])


In [ ]:
filtered = filtered.map(masks2clouds)

## Question 5.3.5
Display result on a Map